In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import scipy
import zipfile

device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class ConvBasic(nn.Module):
    def __init__(self):
        super(ConvBasic, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=2)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(6, 18, kernel_size=5, stride = 2)
        self.relu2 = nn.ReLU(inplace=True)
        self.hidden1 = nn.Linear(271674, 16)
        self.relu4 = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.relu1(self.conv1(x))
        out = self.relu2(self.conv2(out))
        flat_out = out.reshape((x.shape[0], -1))
        
        flat_out = self.relu4(self.hidden1(flat_out))
        return flat_out #16

class GridObservationMLP(nn.Module):
    def __init__(self):
        super(GridObservationMLP, self).__init__()
        self.hidden1 = nn.Linear(25, 256)
        self.relu1 = nn.ReLU(inplace=False)
        self.hidden2 = nn.Linear(256, 512)
        self.relu2 = nn.ReLU(inplace=False)

    def forward(self, x):
        out = self.relu1(self.hidden1(x.reshape(-1, 25)))
        out = self.relu2(self.hidden2(out))
        return out


# TODO: Possible remove belief and action GRU classes and use nn.GRU directly
# TODO: Check GRU num_layers

# class beliefGRU(nn.Module):
#     def __init__(self):
#         super(beliefGRU, self).__init__()
#         # Check input size
#         self.gru1 = nn.GRU(516, 512, batch_first=True)

#     def forward(self, x):
#         out = self.gru1(x)
#         return out


class actionGRU(nn.Module):
    def __init__(self):
        super(actionGRU, self).__init__()
        # Check input size
        self.hid = nn.Linear(4, 16)
        self.rel = nn.ReLU(inplace=False)
        self.gru1 = nn.GRUCell(32, 512)

    def forward(self, image, action, h0):
        act_emb = self.rel(self.hid(action))
        x = torch.cat((image, act_emb), dim=1)
        x = x.reshape((h0.shape[0], -1))
        out = self.gru1(x, h0)
        return out


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # Check input size
        self.hidden1 = nn.Linear(512, 256)
        self.relu1 = nn.ReLU()
        self.hidden2 = nn.Linear(256, 64)
        self.relu2 = nn.ReLU()
        self.hidden3 = nn.Linear(64, 4)

    def forward(self, x):
        out = self.relu1(self.hidden1(x))
        out = self.relu2(self.hidden2(out))
        out = self.hidden3(out)
        return out



class evalMLP(nn.Module):
    def __init__(self, grid_dims):
        super(evalMLP, self).__init__()
        self.x_size, self.y_size = grid_dims
        # TODO: init input size - b_t + grid_size[0]*grid_size[1]
        # TODO: change size to accomodate orientation
        self.hidden1 = nn.Linear(512,  300)
        self.relu1 = nn.ReLU()
        self.hidden3 = nn.Linear(300, 4)
        self.hidden4 = nn.Linear(300, self.x_size * self.y_size)

    def forward(self, x):
        out = self.relu1(self.hidden1(x))
        out_1 = self.hidden3(out)
        out_2 = self.hidden4(out)
        return out_2, out_1

In [5]:
coords = [0.23432111740112305, 0.9721060991287231, 0.9491256475448608, -0.3147679567337036]
image = cv2.imread('Vrnet/2720.jpg')
image = image.transpose(2, 0, 1)
image_t = torch.Tensor(image).to(device)
coords = torch.Tensor(coords).to(device)
h0 = torch.ones((1, 512)).to(device)

init_conv = ConvBasic().to(device)
init_gru = actionGRU().to(device)
fin_mlp = MLP().to(device)

# test = init_conv(image_t)
# test, h1 = init_gru(test, coords,h0)
# test = fin_mlp(test)
# print(test)

In [6]:
earth_path = 'Vrnet/files/earth_data_file.csv'
wild_path = 'Vrnet/files/wild_data_file.csv'

earth_gym = pd.read_csv(earth_path)
wild_quest = pd.read_csv(wild_path)

In [8]:
from ast import literal_eval

earth_fin_gym = pd.DataFrame(data = {'frame': earth_gym['frame'], 'thumbstick': earth_gym['Thumbstick']})
wild_fin_gym = pd.DataFrame(data = {'frame': wild_quest['frame'], 'thumbstick': wild_quest['Thumbstick']})

earth_fin_gym.dropna(how='any', inplace=True)
wild_fin_gym.dropna(how='any', inplace=True)

earth_fin_gym['thumbstick'] = earth_fin_gym['thumbstick'].apply(lambda x : literal_eval(str(x)))
wild_fin_gym['thumbstick'] = wild_fin_gym['thumbstick'].apply(lambda x : literal_eval(str(x)))

earth_fin_gym[['T1', 'T2', 'T3', 'T4']] = pd.DataFrame(earth_fin_gym['thumbstick'].to_list(), index = earth_fin_gym.index)
earth_fin_gym.drop(columns=['thumbstick'], inplace=True)

wild_fin_gym[['T1', 'T2', 'T3', 'T4']] = pd.DataFrame(wild_fin_gym['thumbstick'].to_list(), index = wild_fin_gym.index)
wild_fin_gym.drop(columns=['thumbstick'], inplace=True)

wild_fin_gym

,frame,T1,T2,T3,T4
0,0,0.0,0.0,0.0,0.0
1,10,0.0,0.0,0.0,0.0
2,20,0.0,0.0,0.0,0.0
3,30,0.0,0.0,0.0,0.0
4,40,0.0,0.0,0.0,0.0
...,...,...,...,...,...
5157,51570,0.0,0.0,0.0,0.0
5158,51580,0.0,0.0,0.0,0.0
5159,51590,0.0,0.0,0.0,0.0
5162,51620,0.0,0.0,0.0,0.0


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

optimizer = torch.optim.Adam(
    set(init_conv.parameters()) | set(init_gru.parameters())
    | set(fin_mlp.parameters()), lr=0.0001)
criterion = torch.nn.MSELoss()

earth_fin_gym_t = torch.Tensor(earth_fin_gym.values).to(device)
earth_fin_gym_t = torch.split(earth_fin_gym_t, 100)

if earth_fin_gym_t[-1].shape[0] != 100:
    earth_fin_gym_t = earth_fin_gym_t[:-1]

train_loader = DataLoader(earth_fin_gym_t, batch_size=10, shuffle=True, )
train_ind = earth_fin_gym['frame']

In [ ]:
test = train_loader.__iter__().__next__()
print(test.shape)

In [ ]:
print(train_ind[1])
print(train_loader.__iter__().__next__())

In [ ]:
import tqdm

torch.cuda.empty_cache()


def train(loader, optimizer, criterion):
    init_gru.train()
    init_conv.train()
    init_gru.train()
    
    total_loss = 0

    prog_bar = tqdm.tqdm(range(len(loader)))
    for batch in loader:
        optimizer.zero_grad()
        h0 = torch.ones((batch.shape[0], 512)).to(device)

        for seq in range(batch.shape[0]-1):
            # (100, 5)
            indices = batch[:, seq, 0]
            
            image_t = [cv2.imread(f'Vrnet/files/Earth_gym/{int(img_ind + 10)}.jpg') for img_ind in indices]
            image_t = np.array(image_t)
            image_t = image_t.transpose(0, 3, 1, 2)
            image_t = torch.Tensor(image_t).to(device)
        
            # plt.imshow(image_t)
            # plt.show()
            # raise

            image_r = init_conv(image_t)
            # print(image_r.shape)
            # print(batch[:, seq, 1:].shape)
            # print(h0.shape)
            h0 = init_gru(image_r, batch[:, seq, 1:], h0)

        fin = fin_mlp(h0)
        y = batch[:, -1, 1:]
        loss = criterion(fin, y)
        loss.backward()
        optimizer.step()

        total_loss += loss

        prog_bar.update(1)
    prog_bar.close()

    return loss / len(loader)

print(train(train_loader, optimizer, criterion))

In [2]:
import torch
import numpy as np

empt = torch.empty(0).to(device)
a = torch.Tensor([[0, 1],
                  [1, 2]]).to(device)
b = torch.Tensor([[3, 4]]).to(device)

test = torch.cat((empt,a))
test = torch.cat((test, a))
test = [str(x) for x in test.tolist()]
test

empt1 = torch.empty(1)
empt1

NameError: name 'device' is not defined